In [1]:
from pyomo.environ import *
import pandapower as pp
import pandapower.networks as pn
import pandas as pd

In [2]:
pth = r'C:\Users\Dell\Documents\GitHub\PINNS_OPF\data\interim'
con = r'\asset_cons.csv'
ess = r'\asset_ess.csv'
gen = r'\asset_gen.csv'
pv  = r'\asset_pv.csv'
bus = r'\bus5.csv'
line= r'\line5.csv'
time= r'\time_slots.csv'

In [22]:
df_con = pd.read_csv(pth+con)
df_ess = pd.read_csv(pth+ess)
df_gen = pd.read_csv(pth+gen)
df_pv  = pd.read_csv(pth+pv)
df_bus = pd.read_csv(pth+bus)
df_line= pd.read_csv(pth+line)
df_time= pd.read_csv(pth+time)


In [5]:
# Network Data
# Buses
Ob = [df_bus.loc[i,'name'] for i in df_bus.index]
Vmin = [df_bus.loc[i,'min_vm_pu'] for i in df_bus.index]
Vmax = [df_bus.loc[i,'max_vm_pu'] for i in df_bus.index]

In [6]:
# Branches
Ol = {(df_line.loc[i, 'from_bus'], df_line.loc[i, 'to_bus']) for i in df_line.index}  # Branch nodes
R = {(df_line.loc[i, 'from_bus'], df_line.loc[i, 'to_bus']): df_line.loc[i, 'r_ohm_per_km']  for i in df_line.index}                                    # Branch PVistance in pu
X = {(df_line.loc[i, 'from_bus'], df_line.loc[i, 'to_bus']): df_line.loc[i, 'x_ohm_per_km'] for i in df_line.index}                                    # Branch reactance in pu
Imax = {(df_line.loc[i, 'from_bus'], df_line.loc[i, 'to_bus']): df_line.loc[i, 'max_i_ka'] for i in df_line.index}

In [7]:
# DGs
Odg = [df_gen.loc[i, 'bus'] for i in df_gen.index]                                      # Nodes with DG
DG_Pmin = {Odg[i]: df_gen.loc[i, 'min_p_mw'] for i in df_gen.index}                          # Minimum active power DG
DG_Pmax = {Odg[i]: df_gen.loc[i, 'max_p_mw'] for i in df_gen.index}                          # Maximum active power DG
DG_fp_min = {Odg[i]: df_gen.loc[i, 'fp_min'] for i in df_gen.index}                      # Minimum power factor DG
DG_ramp_up = {Odg[i]: df_gen.loc[i, 'ramp_up'] for i in df_gen.index}                                # ramp up constraint
DG_ramp_dw = {Odg[i]: df_gen.loc[i, 'ramp_down'] for i in df_gen.index}                                # ramp down constraint


In [9]:
# PV
Opv = [df_pv.loc[i, 'PV_node'] for i in df_pv.index]                                  # Nodes with PV
PV_pf = {Opv[i]: df_pv.loc[i, 'PV_pf'] for i in df_pv.index}                         # Power factor PV
PV_sn = {Opv[i]: df_pv.loc[i, 'PV_sn'] for i in df_pv.index}                         # Per unit subsidy
PV_Pmin = {Opv[i]: df_pv.loc[i, 'PV_min'] for i in df_pv.index}                     # Minimum power PV
PV_Pmax = {Opv[i]: df_pv.loc[i, 'PV_max'] for i in df_pv.index}                     # Maximum power PV


In [23]:
# CONSUMERS
Ocons = [i+1 for i in df_con.index]                              # Nodes with flexible CONSUMERS
CONS_nodes = {Ocons[i]: df_con.loc[i, 'Node_cons'] for i in df_con.index}                     # Minimum power CONS
CONS_Pmin = {Ocons[i]: df_con.loc[i, 'CONS_Pmin'] for i in df_con.index}                     # Minimum power CONS
CONS_Pmax = {Ocons[i]: df_con.loc[i, 'CONS_Pmax'] for i in df_con.index}                     # Maximum power CONS
CONS_an = {Ocons[i]: df_con.loc[i, 'CONS_an'] for i in df_con.index}                     # Instantaneous cost load shifted
CONS_bn = {Ocons[i]: df_con.loc[i, 'CONS_bn'] for i in df_con.index}                     # Extra cost load unsatisfied
PD = {Ocons[i]: df_con.loc[i, 'PDn'] for i in df_con.index}                       # Nominal nodal active demand in pu
QD = {Ocons[i]: df_con.loc[i, 'QDn']  for i in df_con.index}                       # Nominal nodal reactive demand in pu
CONS_pf = {Ocons[i]: df_con.loc[i, 'CONS_pf'] for i in df_con.index}                  # Minimum power factor CONS


In [11]:

# Energy storage systems (ESS)
Oess = [df_ess.loc[i, 'ESS_node'] for i in df_ess.index]                                      # Nodes with ESS
ESS_Pmin = {Oess[i]: df_ess.loc[i, 'ESS_Pmin'] for i in df_ess.index}                      # Minimum power ESS
ESS_Pmax = {Oess[i]: df_ess.loc[i, 'ESS_Pmax'] for i in df_ess.index}                      # Maximum power ESS
ESS_Pnom = {Oess[i]: df_ess.loc[i, 'ESS_Pnom'] for i in df_ess.index}                      # Nominal power ESS
ESS_pf_min = {Oess[i]: df_ess.loc[i, 'ESS_pf_min'] for i in df_ess.index}                  # Minimum power factor ESS
ESS_EC = {Oess[i]: df_ess.loc[i, 'ESS_EC'] for i in df_ess.index}                          # Energy capacity ESS
ESS_SOC_ini = {Oess[i]: df_ess.loc[i, 'ESS_SOC_ini']/100 for i in df_ess.index}                # Initial state of charge ESS
ESS_SOC_end = {Oess[i]: df_ess.loc[i, 'ESS_SOC_end']/100 for i in df_ess.index}                # Initial state of charge ESS
ESS_dn = {Oess[i]: df_ess.loc[i, 'ESS_dn'] for i in df_ess.index}                          # Battery  degradation cost ESS
ESS_SOC_min = {Oess[i]: df_ess.loc[i, 'SOC_min']/100 for i in df_ess.index}                        # Minimum state of charge ESS
ESS_SOC_max = {Oess[i]: df_ess.loc[i, 'SOC_max']/100 for i in df_ess.index} 

In [ ]:
# # Electric vehicles (EV)
# Oev = [i+1 for i in df_ev.index]                                      # Index EVs
# EV_nodes = {i:df_ev.loc[i-1, 'EV_node'] for i in Oev}                                      # Nodes with EVs
# EV_Pmin = {i:df_ev.loc[i-1, 'EV_Pmin'] for i in Oev}                                  # Minimum power EVs
# EV_Pmax = {i:df_ev.loc[i-1, 'EV_Pmax'] for i in Oev}                                # Maximum power EVs
# EV_Pnom = {i:df_ev.loc[i-1, 'EV_Pnom']/ (Snom) for i in Oev}                               # Nominal power EVs
# EV_pf_min = {i:df_ev.loc[i-1, 'EV_pf_min'] for i in Oev}                              # Minimum power factor EVs
# EV_EC = {i:df_ev.loc[i-1, 'EV_EC']/ (Snom) for i in Oev}                                      # Energy capacity EVs
# EV_SOC_ini = {i:df_ev.loc[i-1, 'EV_SOC_ini']/100 for i in Oev}                            # Arrive state of charge
# EV_SOC_end = {i:df_ev.loc[i-1, 'EV_SOC_end']/100 for i in Oev}                            # Departure state of charge
# EV_SOC_min = {i: df_ev.loc[i-1, 'EV_SOC_min']/100 for i in Oev}                        # Minimum state of charge ESS
# EV_SOC_max = {i: df_ev.loc[i-1, 'EV_SOC_max']/100 for i in Oev}                        # Maximum state of charge ESS
# EV_dn = {i:df_ev.loc[i-1, 'EV_dn'] for i in Oev}                                      # Battery  degradation cost
# EV_wn = {i:df_ev.loc[i-1, 'EV_wn'] for i in Oev}                                      # Penalty  cost  for  not  having  their  battery  charged  at  the desired level
# t_arr = {i:df_ev.loc[i-1, 't_arr'] for i in Oev}                                      # Time of arrival
# t_dep = {i:df_ev.loc[i-1, 't_dep'] for i in Oev}                                      # Time of departure


In [12]:
OT = [df_time.loc[i, 'T'] for i in df_time.index]
sc_res = {OT[i]: df_time.loc[i, 'Irrad'] for i in df_time.index}
Delta_t = {OT[i]: df_time.loc[i, 'Delta_t'] for i in df_time.index}
pt = {OT[i]: df_time.loc[i, 'pt'] for i in df_time.index}

In [14]:
PM = {(Ocons[i], OT[t]): df_time['PD_{}_{}'.format(df_con['Node_cons'][i],i)][t] for i in df_con.index for t in df_time.index}
QM = {(Ocons[i], OT[t]): df_time['QD_{}_{}'.format(df_con['Node_cons'][i],i)][t] for i in df_con.index for t in df_time.index}
G =  {(Opv[i], OT[t]): df_time['PV_{}_{}'.format(df_pv['PV_node'][i],i)][t] for i in df_pv.index for t in df_time.index}

In [24]:
model = ConcreteModel()

In [25]:
# Define Sets
model.Ob = Set(initialize=Ob)
model.Ol = Set(initialize=Ol)
model.OT = Set(initialize=OT)
model.Odg = Set(initialize=Odg)
model.Opv = Set(initialize=Opv)
model.Ocons = Set(initialize=Ocons)
model.Oess = Set(initialize=Oess)
#model.Oev = Set(initialize=Oev)
#model.EV_nodes = Param(model.Oev,initialize=EV_nodes, mutable=True)

    source (type: set).  This WILL potentially lead to nondeterministic
    behavior in Pyomo


In [27]:
#- Parameters
# Each resource has upper and lower bounds on active power injection 
# Each resource has power factors
# Generators
model.DG_Pmin = Param(model.Odg, initialize=DG_Pmin, mutable=True)  # Minimum power DGs
model.DG_Pmax = Param(model.Odg, initialize=DG_Pmax, mutable=True)  # Maximum power DGs
model.DG_fp_min = Param(model.Odg, initialize=DG_fp_min, mutable=True)  # Minimum power factor DGs
# PV
model.PV_Pmax = Param(model.Opv,model.OT, initialize=G, mutable=True) # Maximum active power PV
model.PV_Pmin = Param(model.Opv, initialize=PV_Pmin, mutable=True)  # Maximum power PVs
model.PV_pf = Param(model.Opv, initialize=PV_pf, mutable=True)  # Constant power factor PVs
# Consumers
model.CONS_Pmin = Param(model.Ocons, initialize=CONS_Pmin, mutable=True)  # Minimum power CONS
model.CONS_Pmax = Param(model.Ocons, initialize=CONS_Pmax, mutable=True)  # Maximum power CONS
model.CONS_pf = Param(model.Ocons, initialize=CONS_pf, mutable=True)  # Constant power factor CONS
# Power factor for consumers missing - TODO
# Storage
model.ESS_Pmin = Param(model.Oess, initialize=ESS_Pmin, mutable=True)  # Minimum power ESS
model.ESS_Pmax = Param(model.Oess, initialize=ESS_Pmax, mutable=True)  # Maximum power ESS
model.ESS_pf_min = Param(model.Oess, initialize=ESS_pf_min, mutable=True)  # Minimum power factor ESS
# EVs - TODO



    'pyomo.core.base.param.IndexedParam'>) on block unknown with a new
    Component (type=<class 'pyomo.core.base.param.IndexedParam'>). This is
    usually indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    'pyomo.core.base.param.IndexedParam'>) on block unknown with a new
    Component (type=<class 'pyomo.core.base.param.IndexedParam'>). This is
    usually indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    'pyomo.core.base.param.IndexedParam'>) on block unknown with a new
    Component (type=<class 'pyomo.core.base.param.IndexedParam'>). This is
    usually indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    'pyomo.core.base.param.IndexedParam'>) on block unknown with a new
    Component (type=<class 'pyomo.core.base.param.IndexedParam'>). This is
    usually indicative of a modelling err

RuntimeError: Cannot add component 'PV_Pmax_index' (type <class 'pyomo.core.base.set.SetProduct_OrderedSet'>) to block 'unknown': a component by that name (type <class 'pyomo.core.base.set.SetProduct_OrderedSet'>) is already defined.

In [137]:
# A resource can be dispatched at any level given descision variable x_{n,t} - Variables
# Generatos
model.dg_x = Var(model.Odg, model.OT, initialize=1.0, within=NonNegativeReals)    # Multiplier demands Generators (0,1)
# Consumers
model.cons_x = Var(model.Ocons, model.OT, initialize=1.0, within=NonNegativeReals)    # Multiplier demands CONSUMERS (0,1)
# PV
model.pv_x = Var(model.Opv, model.OT, initialize=1.0, within=NonNegativeReals)    # Multiplier demands PV (0,1)
# Storage
model.ess_x = Var(model.Oess, model.OT, initialize=1.0, within=NonNegativeReals)    # Multiplier demands PV (0,1)
# EVs - TODO


In [ ]:
# Create rules for active - reactive power
#####################
# ----- DGs -------- #

def DG_constr_active_power_rule(model, i, t):
    return (model.DG_Pmin[i], model.Pdg[i,t], model.DG_Pmax[i])
model.DG_constr_active_power = Constraint(model.Odg, model.OT,
                                    rule=DG_constr_active_power_rule)  # Minimum, Maximum power from DG

def DG_constr_reactive_power_rule(model, i, t):
    return (-model.Pdg[i,t]*tan(acos(model.DG_fp_min[i])) <= model.Qdg[i,t])
model.DG_constr_reactive_power = Constraint(model.Odg, model.OT,
                                    rule=DG_constr_reactive_power_rule)  # Minimum, Maximum power from DG

def DG_constr_reactive_power_rule_2(model, i, t):
    return (model.Qdg[i,t] <= model.Pdg[i,t]*tan(acos(model.DG_fp_min[i])))
    model.DG_constr_reactive_power_2 = Constraint(model.Odg, model.OT,
                                       rule=DG_constr_reactive_power_rule_2)  # Minimum, Maximum power from DG   

In [ ]:
#####################
# ----- ESS -------- #
def ESS_constr_active_power_rule(model, i, t):
    return (model.ESS_Pmin[i]*model.ESS_Pnom[i], model.Pess[i,t], model.ESS_Pmax[i]*model.ESS_Pnom[i])
model.ESS_constr_active_power = Constraint(model.Oess, model.OT,
                                    rule=ESS_constr_active_power_rule)  # Minimum, Maximum power from ESS

def ESS_constr_reactive_power_rule(model, i, t):
    return (-model.Pess[i,t]*tan(acos(model.ESS_pf_min[i])) <= model.Qess[i,t])
model.ESS_constr_reactive_power = Constraint(model.Oess, model.OT,
                                    rule=ESS_constr_reactive_power_rule)  # Minimum, Maximum reactive power from ESS

def ESS_constr_reactive_power_rule_2(model, i, t):
    return (model.Qess[i,t] == model.Pess[i,t]*tan(acos(model.ESS_pf_min[i])))
model.ESS_constr_reactive_power_2 = Constraint(model.Oess, model.OT,
                                    rule=ESS_constr_reactive_power_rule_2)  # Minimum, Maximum reactive power from ESS


In [94]:
# Define Parameters
#model.Vnom = Param(initialize=Vnom, mutable=True)   # Base voltage magnitude
#model.Snom = Param(initialize=Snom, mutable=True)   # Base apparent power
model.Vmin = Param(initialize=Vmin, mutable=True)   # Minimum voltage magnitude
model.Vmax = Param(initialize=Vmax, mutable=True)   # Maximum voltage magnitude
#model.Tb = Param(model.Ob, initialize=Tb, mutable=True) # Type of bus (1 - Slack, 0 - Load)
model.PD = Param(model.Ocons, initialize=PD, mutable=True) # Active power bus demand
model.QD = Param(model.Ocons, initialize=QD, mutable=True) # Reactive power bus demand
model.R = Param(model.Ol, initialize=R, mutable=True) # Line resistance
model.X = Param(model.Ol, initialize=X, mutable=True) # Line reactance
model.Imax = Param(model.Ol, initialize=Imax, mutable=True) # Maximum current magnitude
# model.sc = Param(model.OT, initialize=sc, mutable=True) # Load time scale
model.PM = Param(model.Ocons,model.OT, initialize=PM, mutable=True) # Active power bus demand
model.QM = Param(model.Ocons,model.OT, initialize=QM, mutable=True) # Reactive power bus demand
model.G = Param(model.Opv,model.OT, initialize=G, mutable=True) # Maximum active power bus demand

model.CONS_nodes = Param(model.Ocons, initialize=CONS_nodes, mutable=True) # Reactive power bus demand

In [95]:
# DGs
#model.DG_Pnom = Param(model.Odg, initialize=DG_Pnom, mutable=True)  # Nominal power DGs
model.DG_Pmin = Param(model.Odg, initialize=DG_Pmin, mutable=True)  # Minimum power DGs
model.DG_Pmax = Param(model.Odg, initialize=DG_Pmax, mutable=True)  # Maximum power DGs
model.DG_fp_min = Param(model.Odg, initialize=DG_fp_min, mutable=True)  # Minimum power factor DGs
model.DG_ramp_up = Param(model.Odg, initialize=DG_ramp_up, mutable=True)  # Constant coefficient cost DGs
model.DG_ramp_dw = Param(model.Odg, initialize=DG_ramp_dw, mutable=True)  # Constant coefficient cost DGs
#model.Pdg_ini = Param(model.Odg, initialize=[15,40,10], mutable=True)  # Constant coefficient cost DGs

In [96]:
# PV
#model.RES_Pnom = Param(model.Ores, initialize=RES_Pnom, mutable=True)  # Nominal power RES
model.PV_pf = Param(model.OPV, initialize=PV_pf, mutable=True)  # Constant power factor RES
model.PV_sn = Param(model.OPV, initialize=PV_sn, mutable=True)  # Compensation RES
#model.RES_sc = Param(model.OT, initialize=sc, mutable=True)  # Scale factor RES per period of time
model.PV_Pmax = Param(model.OPV, initialize=PV_Pmax, mutable=True)  # Compensation RES
model.PV_Pmin = Param(model.OPV, initialize=PV_Pmin, mutable=True)  # Compensation RES


In [97]:
# CONS
model.CONS_Pmin = Param(model.Ocons, initialize=CONS_Pmin, mutable=True)  # Minimum power CONS
model.CONS_an = Param(model.Ocons, initialize=CONS_an, mutable=True)  # quadratic component cost CONS
model.CONS_bn = Param(model.Ocons, initialize=CONS_bn, mutable=True)  # constant component cost CONS
model.CONS_Pmax = Param(model.Ocons, initialize=CONS_Pmax, mutable=True)  # Maximum power CONS


In [98]:
# ESS
model.ESS_Pmax = Param(model.Oess, initialize=ESS_Pmax, mutable=True)  # Maximum power ESS
model.ESS_dn = Param(model.Oess, initialize=ESS_dn, mutable=True)  # Depretiation cost ESS
model.ESS_EC = Param(model.Oess, initialize=ESS_EC, mutable=True)  # Energy capacity ESS
model.ESS_Pmin = Param(model.Oess, initialize=ESS_Pmin, mutable=True)  # Minimum power ESS
model.ESS_Pnom = Param(model.Oess, initialize=ESS_Pnom, mutable=True)  # Nominal power ESS
model.ESS_SOC_end = Param(model.Oess, initialize=ESS_SOC_end, mutable=True)  # Final state of charge ESS
model.ESS_SOC_ini = Param(model.Oess, initialize=ESS_SOC_ini, mutable=True)  # Initial state of charge ESS
model.ESS_pf_min = Param(model.Oess, initialize=ESS_pf_min, mutable=True)  # Minimum power factor ESS
model.ESS_SOC_min = Param(model.Oess, initialize=ESS_SOC_min, mutable=True)  # Minimum state of charge ESS
model.ESS_SOC_max = Param(model.Oess, initialize=ESS_SOC_max, mutable=True)  # MAximum state of charge ESS


In [99]:
# # EVs
# model.EV_SOC_end = Param(model.Oev, initialize=EV_SOC_end, mutable=True)  # Departure SOC EV
# model.EV_SOC_ini = Param(model.Oev, initialize=EV_SOC_ini, mutable=True)  # Arrival SOC EV
# model.EV_wn = Param(model.Oev, initialize=EV_wn, mutable=True)  # Cost EV
# model.EV_pf_min = Param(model.Oev, initialize=EV_pf_min, mutable=True)  # Minimum power factor EV
# model.EV_dn = Param(model.Oev, initialize=EV_dn, mutable=True)  # Cost  EV
# model.EV_Pmax = Param(model.Oev, initialize=EV_Pmax, mutable=True)  # Maximum power EV
# model.EV_Pmin = Param(model.Oev, initialize=EV_Pmin, mutable=True)  # Minimum power EV
# model.EV_Pnom = Param(model.Oev, initialize=EV_Pnom, mutable=True)  # Nominal power EV
# model.EV_EC = Param(model.Oev, initialize=EV_EC, mutable=True)  # Energy capacity EV
# model.EV_SOC_min = Param(model.Oev, initialize=EV_SOC_min, mutable=True)  # Energy capacity EV
# model.EV_SOC_max = Param(model.Oev, initialize=EV_SOC_max, mutable=True)  # Energy capacity EV
# model.t_arr = Param(model.Oev, initialize=t_arr, mutable=True)  # Energy capacity EV
# model.t_dep = Param(model.Oev, initialize=t_dep, mutable=True)  # Energy capacity EV


In [100]:
model.Delta_t = Param(model.OT, initialize=Delta_t, mutable=True)  # Time interval
model.pt = Param(model.OT, initialize=pt, mutable=True)  # Wholesale market price

In [101]:
def R_init_rule(model, i,j):
    return (model.R[i,j])
model.RM = Param(model.Ol, initialize= R_init_rule)

def X_init_rule(model, i,j):
    return (model.X[i,j])
model.XM = Param(model.Ol, initialize= X_init_rule)


In [102]:
# Rules for PV
def P_pv_init_rule(model,i,t):
    return (model.G[i,t])
model.PPV = Param(model.OPV, model.OT, initialize= P_pv_init_rule) # Active power per period of time

def Q_pv_init_rule(model,i,t):
    return (model.PPV[i,t] * tan(acos(model.PV_pf[i])))
model.QRES = Param(model.OPV, model.OT, initialize= Q_pv_init_rule) # Reactive power per period of time




In [103]:
# Define Variables
model.P = Var(model.Ol, model.OT, initialize=0) # Acive power flowing in lines
model.Q = Var(model.Ol, model.OT, initialize=0) # Reacive power flowing in lines
model.I  = Var(model.Ol, model.OT, initialize=0) # Current of lines
#model.cf = Var(model.Ob, model.OT, initialize=0.0)
#model.rhoV_slack_max = Var(model.Ob, model.OT, initialize=0.0, within=NonNegativeReals)
#model.rhoV_slack_min = Var(model.Ob, model.OT, initialize=0.0, within=NonNegativeReals)
#model.rhoI_slack_max = Var(model.Ol, model.OT, initialize=0.0, within=NonNegativeReals)

model.Pdg = Var(model.Odg, model.OT, initialize=0.0, within=NonNegativeReals)    # Active power from DG
model.Qdg = Var(model.Odg, model.OT, initialize=0.0)    # Reactive power from DG
model.DG_cost = Var(model.Odg, initialize=1.0)    # Reactive power from DG


model.cons_x = Var(model.Ocons, model.OT, initialize=1.0, within=NonNegativeReals)    # Multiplier demands CONSUMERS (0,1)
model.CONS_cost = Var(model.Ob, initialize=1.0)    # Multiplier demands CONSUMERS (0,1)

model.pv_x = Var(model.OPV, model.OT, initialize=1.0, within=NonNegativeReals)    # Multiplier demands CONSUMERS (0,1)
model.pv_cost = Var(model.OPV, initialize=1.0)    # Multiplier demands CONSUMERS (0,1)



model.Pess = Var(model.Oess,model.OT, initialize=0.0)    # Active power from ESS
model.Qess = Var(model.Oess,model.OT, initialize=0.0)    # Reactive power from ESS
model.SOC = Var(model.Oess,model.OT, initialize=1.0)    # Reactive power from ESS
model.ESS_cost = Var(model.Oess, initialize=1.0)    # Reactive power from ESS

#model.Pev = Var(model.Oev, model.OT, initialize=0.0)    # Active power from EV
#model.Qev = Var(model.Oev, model.OT, initialize=0.0)    # Active power from EV
#model.SOC_EV = Var(model.Oev, model.OT, initialize=0.0)    # SOC from EV
#model.E_non_served = Var(model.Oev, initialize=0.0, within=NonNegativeReals)    # SOC from EV
#model.EV_cost = Var(model.Oev, initialize=1.0)    # SOC from EV




In [104]:
#####################
# ----- DGs -------- #

def DG_constr_active_power_rule(model, i, t):
    return (model.DG_Pmin[i], model.Pdg[i,t], model.DG_Pmax[i])
model.DG_constr_active_power = Constraint(model.Odg, model.OT,
                                    rule=DG_constr_active_power_rule)  # Minimum, Maximum power from DG

def DG_constr_reactive_power_rule(model, i, t):
    return (-model.Pdg[i,t]*tan(acos(model.DG_fp_min[i])) <= model.Qdg[i,t])
model.DG_constr_reactive_power = Constraint(model.Odg, model.OT,
                                    rule=DG_constr_reactive_power_rule)  # Minimum, Maximum power from DG

def DG_constr_reactive_power_rule_2(model, i, t):
    return (model.Qdg[i,t] <= model.Pdg[i,t]*tan(acos(model.DG_fp_min[i])))
model.DG_constr_reactive_power_2 = Constraint(model.Odg, model.OT,
                                    rule=DG_constr_reactive_power_rule_2)  # Minimum, Maximum power from DG

def DG_ramp_rule(model, i, t):
    if t > 1:
        return (model.DG_ramp_dw[i]*model.DG_Pmax[i], model.Pdg[i, t] - model.Pdg[i, t-1],model.DG_ramp_up[i]*model.DG_Pmax[i])
    else:
        return (model.DG_ramp_dw[i] * model.DG_Pmax[i], model.Pdg[i, t] - model.Pdg_ini[i],
                model.DG_ramp_up[i] * model.DG_Pmax[i])
model.DG_ramp = Constraint(model.Odg,model.OT,
                            rule=DG_ramp_rule)  # Minimum, Maximum power from DG



ERROR: Rule failed when generating expression for Constraint DG_ramp with
    index (0, 1): AttributeError: 'ConcreteModel' object has no attribute
    'Pdg_ini'
ERROR: Constructing component 'DG_ramp' from data=None failed: AttributeError:
    'ConcreteModel' object has no attribute 'Pdg_ini'


AttributeError: 'ConcreteModel' object has no attribute 'Pdg_ini'

In [105]:
    #####################
    # ----- ESS -------- #
    def ESS_constr_active_power_rule(model, i, t):
        return (model.ESS_Pmin[i]*model.ESS_Pnom[i], model.Pess[i,t], model.ESS_Pmax[i]*model.ESS_Pnom[i])
    model.ESS_constr_active_power = Constraint(model.Oess, model.OT,
                                       rule=ESS_constr_active_power_rule)  # Minimum, Maximum power from ESS

    def ESS_constr_reactive_power_rule(model, i, t):
        return (-model.Pess[i,t]*tan(acos(model.ESS_pf_min[i])) <= model.Qess[i,t])
    model.ESS_constr_reactive_power = Constraint(model.Oess, model.OT,
                                       rule=ESS_constr_reactive_power_rule)  # Minimum, Maximum reactive power from ESS

    def ESS_constr_reactive_power_rule_2(model, i, t):
        return (model.Qess[i,t] <= model.Pess[i,t]*tan(acos(model.ESS_pf_min[i])))
    model.ESS_constr_reactive_power_2 = Constraint(model.Oess, model.OT,
                                       rule=ESS_constr_reactive_power_rule_2)  # Minimum, Maximum reactive power from ESS


In [106]:
def ESS_constr_SOC_1_rule(model, i, t):
    if t == 1:
        return (model.SOC[i,t] == model.ESS_SOC_ini[i] - model.Delta_t[t]/model.ESS_EC[i]*model.Pess[i,t])
    else:
        return (model.SOC[i,t] == model.SOC[i,t-1] - model.Delta_t[t]/model.ESS_EC[i]*model.Pess[i,t])
model.ESS_constr_SOC_1 = Constraint(model.Oess, model.OT,
                                    rule=ESS_constr_SOC_1_rule)  # Minimum, Maximum reactive power from ESS

def ESS_constr_SOC_2_rule(model, i, t):
    return (model.ESS_SOC_min[i], model.SOC[i,t], model.ESS_SOC_max[i])
model.ESS_constr_SOC_2 = Constraint(model.Oess, model.OT,
                                    rule=ESS_constr_SOC_2_rule)  # Minimum, Maximum reactive power from ESS

def ESS_constr_SOC_departure_rule(model, i):
    return (model.ESS_SOC_ini[i] - sum(model.Delta_t[t]/model.ESS_EC[i]*model.Pess[i,t] for t in model.OT) >= model.ESS_SOC_end[i])
model.ESS_constr_SOC_departure = Constraint(model.Oess,
                                    rule=ESS_constr_SOC_departure_rule)  # Minimum, Maximum reactive power from ESS

def ESS_cost_rule(model, i):
    return (model.ESS_cost[i] == sum(-model.ESS_dn[i]*(model.Pess[i,t])/model.ESS_EC[i] - model.pt[t]*model.Pess[i,t] for t in model.OT))
model.ESS_cf = Constraint(model.Oess,
                                    rule=ESS_cost_rule)  # Minimum, Maximum reactive power from ESS



In [ ]:

#####################
# ----- EVs -------- #
def EV_constr_active_power_rule(model, i, t):
    if model.t_arr[i].value <= t <= model.t_dep[i].value:
        return (model.EV_Pmin[i]*model.EV_Pnom[i], model.Pev[i,t], model.EV_Pmax[i]*model.EV_Pnom[i])
    else:
        return (model.Pev[i, t] == 0.0)
model.EV_constr_active_power = Constraint(model.Oev, model.OT,
                                    rule=EV_constr_active_power_rule)  # Minimum, Maximum power from ESS

def EV_constr_reactive_power_rule(model, i, t):
    return (-model.Pev[i,t]*tan(acos(model.EV_pf_min[i])) <= model.Qev[i,t])
model.EV_constr_reactive_power = Constraint(model.Oev, model.OT,
                                    rule=EV_constr_reactive_power_rule)  # Minimum, Maximum reactive power from ESS

def EV_constr_reactive_power_rule_2(model, i, t):
    return (model.Qev[i,t] <= model.Pev[i,t]*tan(acos(model.EV_pf_min[i])))
model.EV_constr_reactive_power_2 = Constraint(model.Oev, model.OT,
                                    rule=EV_constr_reactive_power_rule_2)  # Minimum, Maximum reactive power from ESS
#
#
def EV_constr_SOC_1_rule(model, i, t):
    if t <= model.t_arr[i].value:
        return (model.SOC_EV[i,t] == model.EV_SOC_ini[i] - model.Delta_t[t]/model.EV_EC[i]*model.Pev[i,t])
    else:
        return (model.SOC_EV[i,t] == model.SOC_EV[i,t-1] - model.Delta_t[t]/model.EV_EC[i]*model.Pev[i,t])
model.EV_constr_SOC_1 = Constraint(model.Oev, model.OT,
                                    rule=EV_constr_SOC_1_rule)  # Minimum, Maximum reactive power from ESS

def EV_constr_SOC_2_rule(model, i, t):
    return (model.EV_SOC_min[i], model.SOC_EV[i,t], model.EV_SOC_max[i])
model.EV_constr_SOC_2 = Constraint(model.Oev, model.OT,
                                    rule=EV_constr_SOC_2_rule)  # Minimum, Maximum reactive power from ESS

def EV_constr_SOC_departure_rule(model, i):
    return (model.SOC_EV[i,model.t_dep[i].value] + model.E_non_served[i] >= model.EV_SOC_end[i]) #
model.EV_constr_SOC_departure = Constraint(model.Oev,
                                    rule=EV_constr_SOC_departure_rule)  # Minimum, Maximum reactive power from ESS
#
def EV_cost_rule(model, i):
    return (model.EV_cost[i] == sum(-model.EV_dn[i]*(model.Pev[i,t])/model.EV_EC[i] - model.pt[t]*model.Pev[i,t] for t in model.OT)
            + model.EV_wn[i]* (model.E_non_served[i]))
model.EV_cf = Constraint(model.Oev,
                                    rule=EV_cost_rule)  # Minimum, Maximum reactive power from ESS



In [109]:
import numpy as np

In [121]:
model.__dict__

{'_ctypes': {pyomo.core.base.set.Set: [0, 92, 31],
  pyomo.core.base.param.Param: [7, 51, 40],
  pyomo.core.base.var.Var: [53, 75, 14],
  pyomo.core.base.constraint.Constraint: [77, 95, 11]},
 '_decl': {'Ob': 0,
  'Ol': 1,
  'OT': 2,
  'Odg': 3,
  'OPV': 4,
  'Ocons': 5,
  'Oess': 6,
  'Vmin': 7,
  'Vmax': 8,
  'PD': 9,
  'QD': 10,
  'R': 11,
  'X': 12,
  'Imax': 13,
  'PM_index': 14,
  'PM': 15,
  'QM_index': 16,
  'QM': 17,
  'G_index': 18,
  'G': 19,
  'CONS_nodes': 20,
  'DG_Pmin': 21,
  'DG_Pmax': 22,
  'DG_fp_min': 23,
  'DG_ramp_up': 24,
  'DG_ramp_dw': 25,
  'PV_pf': 26,
  'PV_sn': 27,
  'PV_Pmax': 28,
  'PV_Pmin': 29,
  'CONS_Pmin': 30,
  'CONS_an': 31,
  'CONS_bn': 32,
  'CONS_Pmax': 33,
  'ESS_Pmax': 34,
  'ESS_dn': 35,
  'ESS_EC': 36,
  'ESS_Pmin': 37,
  'ESS_Pnom': 38,
  'ESS_SOC_end': 39,
  'ESS_SOC_ini': 40,
  'ESS_pf_min': 41,
  'ESS_SOC_min': 42,
  'ESS_SOC_max': 43,
  'Delta_t': 44,
  'pt': 45,
  'RM': 46,
  'XM': 47,
  'PPV_index': 48,
  'PPV': 49,
  'QRES_index': 50

In [126]:
solver = SolverFactory('gurobi')  # couenne

# Solve
results = solver.solve(model)
model.display()

ValueError: ERROR: No objectives defined for input model. Cannot write legal LP file.

In [117]:
suma = 'dual_file.csv'
dash1 = '-' * 100
dash2 = '-' * 30

with open(suma, 'w') as f:
    print("T,", file=f, end="")
    for i in result.Ob:
        print("lambdaP_{},".format(i), file=f, end="")
    for i in result.Ob:
        print("lambdaQ_{}".format(i), file=f, end="")
        if i != (result.Ob[-1]):
            print(",", file=f, end="")
        else:
            print("", file=f)

NameError: name 'result' is not defined

In [116]:
Odg

[0, 2, 4]